In [2]:
from pathlib import Path  # pathlib: module, Path: class. Checking if a path exist
from typing import Optional, List, Dict, Tuple  # typing: support for type hint
import pandas as pd
import numpy as np
import collections  # This module contains different datatype to process the data: dict, list, set, and tuple.
from pprint import pprint  # pprint.pprint() can use when you need to examine the structure of a large or complex
# data structure. this output reveals more readable and structured way.
from who_member_states import WHO_MEMBER_STATES

In [3]:

__all__ = ['select_df']  # only import 'select_df'


# test data should <10 MB

def select_df(df: pd.DataFrame,
              rename_mapping: Dict[str, str] = None,
              column_drop: Optional[List[str]] = None,  # column_drop (param) is an optional list of string. Optional
              # type is from Typing. default is None (no drop any column). If provide column_drop, must be ['xxx'] (
              # list of string)
              year: int = 2000,
              save_path: Optional[Path] = None,
              drop_na: Optional[List[str]] = None, ) -> pd.DataFrame:
    """
    dataframe modification and save as another file

    :param df: input dataframe
    :param rename_mapping: both df have countries but the header is different
    :param column_drop: drop the column(s) that are not informative
    :param year: pick up the data that larger than which year
    :param save_path: path for saving the modified dataframe
    :param drop_na: specify if drop the na-existing column name
    :return: modified df
    """

    df = df.copy()  # The copy() method returns a copy of the DataFrame. By default, the copy is a "deep copy"
    # meaning that any changes  made in the original DataFrame will NOT be reflected in the copy. 新跑出來的df不會影響最原始的df

    if rename_mapping is not None:
        df = df.rename(columns=rename_mapping)
    if column_drop is not None:  # if drop specific column,df need to drop the option
        df = df.drop(columns=column_drop)  # df 的column = column after drop, column_drop: Optional[List[str]]

    year_mask: pd.Series[bool] = df['Year'] >= year  # type is list of bool; compared df["year"] whether larger than
    # default year (year: int = 2000)
#    entity_mask: List[bool] = [country in WHO_MEMBER_STATES for country in df['Country Name']]  # check countries whether
    # in df['Entity'] also in WHO_MEMBER_STATES. If Yes =True
    modified_df: pd.DataFrame = df[year_mask].reset_index(
        drop=True)  # create a new df that only meet both
    # year_mask and entity_mask. df.reset_index(drop = True) means new index created, old index don't added in new df.

    if drop_na is not None:  # drop rows with missing values ('NaN') from df
        try:
            modified_df.dropna(subset=drop_na, inplace=True)
            # inplace = True means that the original df will be modified and no copy will be made.; But, if inplace =
            # False, df will still show the initial one. subset = drop_na means drop in specific place you set.
        except KeyError as e:
            raise ValueError(f'{e} not in the dataframe, should be one of the {modified_df.columns.tolist()}')  # If
            # typed wrong, show the list which should be dropped.
    if save_path is not None:
        modified_df.to_excel(save_path)

    return modified_df


In [4]:
if __name__ == '__main__':
    raw_who_cvd_df = pd.read_csv('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/raw '
                                 'data/WHOMortalityDatabase_Deaths_sex_age_a_country_area_year-Cardiovascular '
                                 'diseases_7th February 2023.csv')
column_drop = ['Age group code', 'Unnamed: 12']
na_header = ['Number',
             'Percentage of cause-specific deaths out of total deaths',
             'Death rate per 100 000 population']
save_path = (
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHO_Cardiovascular_Disease_Mortality_Database.xlsx')
who_cvd_df = select_df(raw_who_cvd_df, column_drop=column_drop, drop_na=na_header,
                       save_path=save_path)

In [5]:
number_of_countries=raw_who_cvd_df['Country Name'].unique()
print(len(number_of_countries),raw_who_cvd_df.shape, raw_who_cvd_df.columns.tolist())
print(len(who_cvd_df['Country Name'].unique()),who_cvd_df.shape,who_cvd_df.columns.tolist())


114 (297066, 13) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'Sex', 'Age group code', 'Age Group', 'Number', 'Percentage of cause-specific deaths out of total deaths', 'Age-standardized death rate per 100 000 standard population', 'Death rate per 100 000 population', 'Unnamed: 12']
111 (111401, 11) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'Sex', 'Age Group', 'Number', 'Percentage of cause-specific deaths out of total deaths', 'Age-standardized death rate per 100 000 standard population', 'Death rate per 100 000 population']


In [6]:
def preprocess_cvd(df: pd.DataFrame,
                   drop_na: Optional[List[str]] = None,
                   save_path: Optional[Path] = None) -> pd.DataFrame:
    """
    Dataframe of WHO_Cardiovascular_Disease_Mortality_Database.xlsx need to modify:
    - grouping if set kwarg `grouping_age` as true  # kwarg : keyword arguments
    - Calculate Total number of death
    - only left age >15 years old
    - save as another dataframe

    :param df: input dataframe (WHO_Cardiovascular_Disease_Mortality_Database.xlsx)
    :param drop_na: drop Age_Group <15 year-old
    :param save_path: save modified dataframe to another excel
    :return: df
    """
    
    # Sum of number of death in each age group
    numbers = df['Number']
    percentage = df['Percentage of cause-specific deaths out of total deaths']
    df["Total Number of Cause-Specific Deaths"] = numbers * 100 / percentage
    total_number_of_death = df["Total Number of Cause-Specific Deaths"]
    mask_nan = np.isnan(total_number_of_death)  # type: pd.Series[bool] # if value is NaN, NaN = True

    df.loc[mask_nan, "Total Number of Cause-Specific Deaths"] = 0  # search location of df. if index ('Total number of death') is
    # NaN, change NaN to 0. ( if no.loc :SettingWithCopyWarning: A value is trying to be set on a copy of a slice
    # from a DataFrame
    df["Total Number of Cause-Specific Deaths"] = df["Total Number of Cause-Specific Deaths"].astype(int)  # astype can cast/change multiple types (
    # change type to int)
    if drop_na is not None:  # drop rows with missing values ('NaN') from df
        df = df.mask(df['Age Group'].isin(['[0]', '[1-4]', '[5-9]', '[10-14]']), np.nan)
        df.dropna(subset=['Age Group'], inplace=True)
    
    if save_path is not None:
        df.to_excel(save_path)
    return df


In [7]:
who_cvd_df= pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHO_Cardiovascular_Disease_Mortality_Database.xlsx')
# “xlrd” supports old-style Excel files (.xls).“openpyxl” supports newer Excel file formats.
drop_na =['Age Group']
save_path=('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHO_Cardiovascular_Disease_Mortality_Database_preprocess.xlsx')
who_cvd_df_preprocess = preprocess_cvd(who_cvd_df,drop_na= drop_na,save_path=save_path)  # assign a who_cvd_df after preprocess_cvd
print(len(who_cvd_df_preprocess['Country Name'].unique()),who_cvd_df_preprocess.shape,who_cvd_df_preprocess.columns.tolist())

111 (89218, 13) ['Unnamed: 0', 'Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'Sex', 'Age Group', 'Number', 'Percentage of cause-specific deaths out of total deaths', 'Age-standardized death rate per 100 000 standard population', 'Death rate per 100 000 population', 'Total Number of Cause-Specific Deaths']


In [8]:
def create_age_grouping(df: pd.DataFrame,
                        save_path: Optional[Path] = None) -> pd.DataFrame:
    """
    Calculate: Total percentage of CVD of total deaths = Sum of number/ Sum of Total number of cause-specific deaths
    * 100 (Male/ Female/ All in each year and country)
    grouping_age: Age groups --> one age group (greater 15 y/o)
    create a new df and save it to excel

    :param df: df after select_df and preprocess_cvd
    :param save: save modified dataframe to another excel
    :return: new df
    """

    if 'Total Number of Cause-Specific Deaths' not in df.columns:
        raise RuntimeError('call preprocess_cvd in advance')

    dy: dict = collections.defaultdict(list)  # defaultdict object in collections. datatype will be dict. Using list
    # as the default_factory to group a sequence of key-value pairs into a dictionary of lists
    group = df.groupby(['Region Code','Region Name','Country Code','Country Name', 'Year', 'Sex'])
    info: List[Tuple] = list(group.groups.keys())  # List[Tuple]: value is a list of tuple[()].looking for the keys in a
    # dict. The 'groups' attribute of the 'groupby' object is always dic type

    for i, it in enumerate(info):  # i = index ( starting from 0) , it = item (Entity, Year, Sex). enumerate can pair
        # index and item
        dy['Region Code'].append(it[0])
        dy['Region Name'].append(it[1]) 
        dy['Country Code'].append(it[2]) 
        dy['Country Name'].append(it[3])  
        dy['Year'].append(it[4])  # Year in [4]
        dy['Sex'].append(it[5])  # Sex in [5]

    numbers = group['Number']
    total_number_of_death = group['Total Number of Cause-Specific Deaths']
    # noinspection PyTypeChecker todo: wt is it?
    dy['Number'] = np.array(numbers.sum())
    dy['Total Number of Cause-Specific Deaths'] = np.array(total_number_of_death .sum())
    dy['Total Percentage of Cause-Specific Deaths Out Of Total Deaths'] = np.array(numbers.sum() / total_number_of_death.sum() * 100)

    _df = pd.DataFrame.from_dict(dy)  # creates a new_df from the dy dictionary.

    # change layout
    sex_values = ['All', 'Female', 'Male']
    new_df = _df.assign(
        All_Number=_df.query("Sex == 'All'")['Number'],
        Female_Number=_df.query("Sex == 'Female'")['Number'],
        Male_Number=_df.query("Sex == 'Male'")['Number'],
        All_Total_Number_of_Cause_Specific_Deaths=_df.query("Sex == 'All'")['Total Number of Cause-Specific Deaths'],
        Female_Total_Number_of_Cause_Specific_Deaths=_df.query("Sex == 'Female'")['Total Number of Cause-Specific Deaths'],
        Male_Total_Number_of_Cause_Specific_Deaths=_df.query("Sex == 'Male'")['Total Number of Cause-Specific Deaths'],
        All_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths=_df.query("Sex == 'All'")['Total Percentage of Cause-Specific Deaths Out Of Total Deaths'],
        Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths=_df.query("Sex == 'Female'")['Total Percentage of Cause-Specific Deaths Out Of Total Deaths'],
        Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths=_df.query("Sex == 'Male'")['Total Percentage of Cause-Specific Deaths Out Of Total Deaths'])
    new_df.reset_index(drop=True, inplace=True)

    new_df = new_df.drop(['Sex', 'Number', 'Total Number of Cause-Specific Deaths', 'Total Percentage of Cause-Specific Deaths Out Of Total Deaths'],
                         axis=1)  # axis = 1: specifies to drop columns

    new_df = new_df.groupby(['Region Code','Region Name','Country Code','Country Name', 'Year']).first().reset_index()  # The first method is then applied to
    # the grouped dataframe, which returns the first row of each group
    if save_path:
        new_df.to_excel(save_path)
    return new_df


In [9]:
who_cvd_df_preprocess = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHO_Cardiovascular_Disease_Mortality_Database_preprocess.xlsx')
who_cvd_df_preprocess= preprocess_cvd(who_cvd_df_preprocess)
save_path = ('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/new_WHO_Cardiovascular_Disease_Mortality_Database.xlsx')
new_df = create_age_grouping(who_cvd_df_preprocess,save_path=save_path)
print(len(new_df['Country Name'].unique()),new_df.shape,new_df.columns.tolist())

111 (1878, 14) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'All_Number', 'Female_Number', 'Male_Number', 'All_Total_Number_of_Cause_Specific_Deaths', 'Female_Total_Number_of_Cause_Specific_Deaths', 'Male_Total_Number_of_Cause_Specific_Deaths', 'All_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths']


In [10]:
def tobacco_layout_modified(df: pd.DataFrame,
                            column_drop: Optional[List[str]] =None,
                            save_path: Optional[Path] = None) -> pd.DataFrame:
    if column_drop is not None:
        df = df.drop(columns=column_drop)
    dy: dict = collections.defaultdict(list)
    group = df.groupby(['Country Name', 'Year', 'Indicator', 'Sex'])
    info: List[Tuple] = list(group.groups.keys())

    for i, it in enumerate(info):
        dy['Country Name'].append(it[0])
        dy['Year'].append(it[1])
        dy['Indicator'].append(it[2])
        dy['Sex'].append(it[3])
        dy['Prevalence'].append(group.get_group(it).Prevalence.mean())

    _df = pd.DataFrame.from_dict(dy)
    sex_values = ['Both sexes', 'Male', 'Female']
    indicator_values = ['Estimate of current tobacco use prevalence (%) (age-standardized rate)',
                        'Estimate of current tobacco smoking prevalence (%) (age-standardized rate)']

    changed_df = _df.assign(
        All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Both sexes'& Indicator =='Estimate of current tobacco use prevalence (%) (age-standardized rate)'")[
            'Prevalence'],
        Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Male'& Indicator =='Estimate of current tobacco use prevalence (%) (age-standardized rate)'")[
            'Prevalence'],
        Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Female'& Indicator =='Estimate of current tobacco use prevalence (%) (age-standardized rate)'")[
            'Prevalence'],

        All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Both sexes'& Indicator =='Estimate of current tobacco smoking prevalence (%) (age-standardized "
            "rate)'")['Prevalence'],
        Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Male'& Indicator =='Estimate of current tobacco smoking prevalence (%) (age-standardized rate)'")[
            'Prevalence'],
        Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Female'& Indicator =='Estimate of current tobacco smoking prevalence (%) (age-standardized rate)'")[
            'Prevalence'])

    changed_df.reset_index(drop=True, inplace=True)

    changed_df = changed_df.drop(['Sex', 'Prevalence', 'Indicator'], axis=1)
    changed_df = changed_df.groupby(['Country Name', 'Year']).first().reset_index()
    print(changed_df)
    if save_path is not None:
        changed_df.to_excel(save_path)
    return changed_df


In [11]:
raw_tobacco_df = pd.read_csv(
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/raw data/Estimate of current tobacco smoking '
    'prevalence(%)(age-standardized rate)_17 Jan 2022.csv')

rename = {'Location': 'Country Name', 'Period': 'Year', 'Dim1': 'Sex', 'First Tooltip': 'Prevalence'}

save_path = '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/Prevalence of Tobacco use_modified.xlsx'
tobacco_df = select_df(raw_tobacco_df, rename_mapping=rename, save_path=save_path)
print(len(raw_tobacco_df['Location'].unique()),raw_tobacco_df.shape,raw_tobacco_df.columns.tolist())
print(len(tobacco_df['Country Name'].unique()),tobacco_df.shape,tobacco_df.columns.tolist())

165 (9351, 5) ['Location', 'Period', 'Indicator', 'Dim1', 'First Tooltip']
165 (9351, 5) ['Country Name', 'Year', 'Indicator', 'Sex', 'Prevalence']


In [12]:
tobacco_df = pd.read_excel(
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/Prevalence of Tobacco use_modified.xlsx')

save_path = '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/Prevalence of Tobacco use_Changed_layout.xlsx'
changed_df = tobacco_layout_modified(tobacco_df, save_path=save_path)

print(len(changed_df['Country Name'].unique()),changed_df.shape,changed_df.columns.tolist())


     Country Name  Year  \
0     Afghanistan  2000   
1     Afghanistan  2005   
2     Afghanistan  2010   
3     Afghanistan  2015   
4     Afghanistan  2018   
...           ...   ...   
1472     Zimbabwe  2018   
1473     Zimbabwe  2019   
1474     Zimbabwe  2020   
1475     Zimbabwe  2023   
1476     Zimbabwe  2025   

      All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate  \
0                                                  36.9                      
1                                                  32.7                      
2                                                  29.1                      
3                                                  26.0                      
4                                                  23.8                      
...                                                 ...                      
1472                                               12.1                      
1473                                               

In [13]:
# merge df1 & df2 test
df1 = new_df
df2 = changed_df

cvd_tobacco = pd.merge(df1, df2, on=['Country Name', 'Year'], how='outer')
cvd_tobacco.fillna(value='NaN',
                   inplace=True)  # inplace = True means that 'value = 'NaN'' will inplace original value in df. 'Nan' can changed what you want to instead of.
cvd_tobacco.to_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/merge_cvd_tobacco.xlsx')

print(len(cvd_tobacco['Country Name'].unique()),cvd_tobacco.shape, cvd_tobacco.columns.tolist())


185 (2895, 20) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'All_Number', 'Female_Number', 'Male_Number', 'All_Total_Number_of_Cause_Specific_Deaths', 'Female_Total_Number_of_Cause_Specific_Deaths', 'Male_Total_Number_of_Cause_Specific_Deaths', 'All_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate', 'Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate', 'Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate', 'All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate', 'Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate', 'Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate']


In [23]:
def format_date(df: pd.DataFrame,
                rename_mapping: Dict[str, str] = None,
                formatted_date: Optional[List[str]] = None,
                save_path: Optional[Path] = None) -> pd.DataFrame:
    if rename_mapping is not None:
        df = df.rename(columns=rename_mapping)
    if formatted_date is not None:
        for col in formatted_date:
            df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
            df[col] = df[col].dt.strftime("%d %b %Y")
    df.fillna(value='Nan', inplace=True)
    if save_path is not None:
        df.to_excel(save_path)
    return df

In [38]:
if __name__ == '__main__':
    df = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/raw data/Signatures and '
                       'Ratifications- UN Treaty Section_08 Feb_2023 .xlsx', engine='openpyxl')
    rename = {'Participant': 'Country Name',
              "Ratification, Acceptance(A), Approval(AA), Formal confirmation(c), Accession(a), Succession(d)": 'Ratification'}
    formatted_date = ['Signature', 'Ratification']
    save_path = '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_date_formatted.xlsx'
    format_date(df, rename_mapping=rename, formatted_date=formatted_date, save_path=save_path)

print(len(who_fctc_date_formatted['Country Name'].unique()),who_fctc_date_formatted.shape,who_fctc_date_formatted.columns.tolist())



188 (188, 3) ['Country Name', 'Signature', 'Ratification']


In [39]:
# merge
df1 = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/merge_cvd_tobacco.xlsx',
                    engine='openpyxl')
df2 = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_date_formatted.xlsx',
                    engine='openpyxl')

signed_df = pd.merge(df1, df2, on=['Country Name'], how='outer')
signed_df.fillna(value='NaN', inplace=True)
signed_df.to_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_signed_date.xlsx')

In [40]:

# drop_missing_data
df = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_signed_date.xlsx')
df = df.dropna(how='any')  # drop the rows having Nan
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0_y'])

df.to_excel(
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_signed_date_no_missingdata.xlsx')


countries = df['Country Name'].unique()
number_of_countries = len(countries)
print(number_of_countries, countries, df.shape)  


82 ['Egypt' 'Mauritius' 'Seychelles' 'South Africa' 'Armenia' 'Azerbaijan'
 'Bahrain' 'Brunei Darussalam' 'Cyprus' 'Georgia'
 'Iran (Islamic Republic of)' 'Iraq' 'Israel' 'Jordan' 'Japan'
 'Kazakhstan' 'Kyrgyzstan' 'Republic of Korea' 'Kuwait' 'Lebanon'
 'Sri Lanka' 'Maldives' 'Philippines' 'Singapore' 'Thailand'
 'Turkmenistan' 'Uzbekistan' 'Argentina' 'Belize' 'Brazil' 'Chile'
 'Colombia' 'Costa Rica' 'Ecuador' 'Guatemala' 'Guyana' 'Mexico' 'Panama'
 'Peru' 'Paraguay' 'El Salvador' 'Albania' 'Austria' 'Belgium' 'Bulgaria'
 'Bosnia and Herzegovina' 'Belarus' 'Switzerland' 'Germany' 'Spain'
 'Estonia' 'Finland' 'France' 'Greece' 'Croatia' 'Hungary' 'Ireland'
 'Iceland' 'Italy' 'Lithuania' 'Luxembourg' 'Latvia' 'Republic of Moldova'
 'Malta' 'Netherlands' 'Norway' 'Poland' 'Portugal' 'Romania'
 'Russian Federation' 'Serbia' 'Slovakia' 'Slovenia' 'Sweden' 'Ukraine'
 'Bahamas' 'Barbados' 'Canada' 'Cuba' 'Jamaica' 'United States of America'
 'Fiji'] (424, 22)


In [50]:
#mask non ratified parties

mask = df['Ratification'] != 'Nan'
df = df[mask]
df = df[df['Ratification'] != 'Nan']
df.to_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH '
            'Dissertation/Data/WHOFCTC_Parties_ratified_date_no_missingdata.xlsx') 
print(df['Country Name'].unique(),len(df['Country Name'].unique()), df.shape)  


['Egypt' 'Mauritius' 'Seychelles' 'South Africa' 'Armenia' 'Azerbaijan'
 'Bahrain' 'Brunei Darussalam' 'Cyprus' 'Georgia'
 'Iran (Islamic Republic of)' 'Iraq' 'Israel' 'Jordan' 'Japan'
 'Kazakhstan' 'Kyrgyzstan' 'Republic of Korea' 'Kuwait' 'Lebanon'
 'Sri Lanka' 'Maldives' 'Philippines' 'Singapore' 'Thailand'
 'Turkmenistan' 'Uzbekistan' 'Belize' 'Brazil' 'Chile' 'Colombia'
 'Costa Rica' 'Ecuador' 'Guatemala' 'Guyana' 'Mexico' 'Panama' 'Peru'
 'Paraguay' 'El Salvador' 'Albania' 'Austria' 'Belgium' 'Bulgaria'
 'Bosnia and Herzegovina' 'Belarus' 'Germany' 'Spain' 'Estonia' 'Finland'
 'France' 'Greece' 'Croatia' 'Hungary' 'Ireland' 'Iceland' 'Italy'
 'Lithuania' 'Luxembourg' 'Latvia' 'Republic of Moldova' 'Malta'
 'Netherlands' 'Norway' 'Poland' 'Portugal' 'Romania' 'Russian Federation'
 'Serbia' 'Slovakia' 'Slovenia' 'Sweden' 'Ukraine' 'Bahamas' 'Barbados'
 'Canada' 'Jamaica' 'Fiji'] 78 (400, 22)


In [ ]:

import matplotlib.pyplot as plt

WHOFCTC_Parties_date = pd.read_excel("/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_date_filter.xlsx")
CVD_Tobacco_ratified_parites = pd.read_excel("/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/CVD_Tobacco_Parties_ratified.xlsx")
WHOFCTC_Parties_date = WHOFCTC_Parties_date[WHOFCTC_Parties_date['Ratification'] != 'Nan']
WHOFCTC_Parties_date['Ratification'] = pd.to_datetime(WHOFCTC_Parties_date['Ratification'], infer_datetime_format='%d/%m/%Y')
WHOFCTC_Parties_date['Year'] = WHOFCTC_Parties_date['Ratification'].dt.year
WHOFCTC_Parties_date

In [ ]:
CVD_Tobacco_ratified_parites = CVD_Tobacco_ratified_parites[CVD_Tobacco_ratified_parites['Country'] == 'Australia']
years = CVD_Tobacco_ratified_parites['Year'].astype(str)
plt.plot(years, CVD_Tobacco_ratified_parites['Male_total_percentage_of_CVD'], label='Total Percentage of CVD in Males')
plt.plot(years, CVD_Tobacco_ratified_parites['Prevalence of current tobacco use, males (% of male adults)'], label='Prevalence of Current Tobacco Use in Males')
plt.xlabel('Year')
plt.ylabel('Percentage/Prevalence')
plt.legend()
plt.xticks(years, rotation=90)
plt.title('CVD and Tobacco Use in Australia')
plt.show()


In [ ]:

def plot_relationship(df: pd.DataFrame,
                      select_country: Optional[List[str]] = None,
                      variable_1: Optional[List[str]] = None,
                      variable_2: Optional[List[str]] = None,
                      x_label: str = None,
                      y_label: str = None,
                      save_path: Optional[Path] = None) -> pd.DataFrame:
    df['Year'] = df['Year'].astype(str)
    if select_country is not None:
        for country in select_country:
            df = df[df['Entity'] == country]
        #df = df[df['Entity'].isin(select_country)]
            if variable_1 is not None:
                plt.plot(df['Year'], df[variable_1[0]], label=variable_1[0])
            if variable_2 is not None:
                plt.plot(df['Year'], df[variable_2[0]], label=variable_2[0])
            if x_label is not None:
                plt.xlabel(x_label)
            if y_label is not None:
                plt.ylabel(y_label)
            plt.legend()
            plt.xticks(df['Year'], rotation=90)

            plt.title(f'CVD Mortality and Prevalence of Tobacco Use in {select_country[0]}')
            plt.show()

    if save_path is not None:
        df.to_excel(save_path)
    return df



In [ ]:
def plot_relationship(df: pd.DataFrame,
                      select_country: Optional[List[str]] = None,
                      variable_1: Optional[List[str]] = None,
                      variable_2: Optional[List[str]] = None,
                      x_label: str = None,
                      y_label: str = None,
                      save_path: Optional[Path] = None) -> pd.DataFrame:
    df['Year'] = df['Year'].astype(str)
    if select_country is not None:
        for country in select_country:
            country_df = df[df['Entity'] == country]
            if variable_1 is not None:
                plt.plot(country_df['Year'], country_df[variable_1[0]], label=variable_1[0])
            if variable_2 is not None:
                plt.plot(country_df['Year'], country_df[variable_2[0]], label=variable_2[0])
            if x_label is not None:
                plt.xlabel(x_label)
            if y_label is not None:
                plt.ylabel(y_label)
            plt.legend()
            plt.xticks(country_df['Year'], rotation=90)

            plt.title(f'CVD Mortality and Prevalence of Tobacco Use in {country}')
            plt.show()

    if save_path is not None:
        df.to_excel(save_path)
    return df
if __name__ == '__main__':
    df= pd.read_excel(
        "/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/CVD_Tobacco_Parties_ratified.xlsx")
    select_country =df['Entity'].unique()
    variable_1 = ['Female_total_percentage_of_CVD']
    variable_2 = ['Prevalence of current tobacco use, females (% of female adults)']
    x_label = 'Year'
    y_label = 'Total Percentage of CVD Mortality & Prevalence of Tobacco Use'
    for country in select_country:
        df = plot_relationship(df, select_country=[country], variable_1=variable_1, variable_2=variable_2,
                           x_label=x_label, y_label=y_label)
